In [1]:
import os
import re
import openai
import pinecone
from time import time, sleep

/Users/farisology/.pyenv/versions/3.11.0/envs/casiaenv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [18]:
# openai.api_key = "--"
# pinecone.init(api_key = "--", environment = 'us-east1-gcp')
# vdb = pinecone.Index("testbed")

def gpt3_embedding(content, engine='text-embedding-ada-002'):
    content = content.encode(encoding='ASCII',errors='ignore').decode()  # fix any UNICODE errors
    response = openai.Embedding.create(input=content,engine=engine)
    vector = response['data'][0]['embedding']  # this is a normal list
    return vector

In [3]:
a= "what plants to grow in Malaysia?"

vector = gpt3_embedding(a)
results = vdb.query(vector=vector, top_k=3, include_values=False, include_metadata=True, namespace="casia_kb")

In [20]:
def save_file(filepath, content):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)

def chadgpt_completion(question, context):
    max_retry = 5
    retry = 0
    preample = """You are casia a wise botanist with vast knowledge in gardening and great servitude to human well-being. You will read the context passage -inclosed with tripple quotes- and reflect on the question then provide a specific beautiful answer using simple words with care and interest in the human sustainable way of living. Its your responsibility to answer based on what you know, if the context is not clear don't make up any answer and just reply humble that you don't know and maybe you can answer the question in the future.

Passage:"""
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-16k",#'gpt-3.5-turbo',
                messages=[
                    {"role": "system", "content": f"{preample}"},
                    {"role": "assistant", "content": f"{context}"},
                    {"role": "user", "content": f"{question}"},
                ],
                temperature=0,
                top_p=1.0, max_tokens=500, frequency_penalty=0.0, presence_penalty=0.0,
            )
            text = response["choices"][0]["message"]["content"].strip()
            text = re.sub('[\r\n]+', '\n', text)
            text = re.sub('[\t ]+', ' ', text)
            filename = '%s_gpt3.txt' % time()
            if not os.path.exists('gpt3_logs'):
                os.makedirs('gpt3_logs')
            save_file('gpt3_logs/%s' % filename, preample +
                      '\n\n==========\n\n' + text)
            return text
        except Exception as oops:
            retry += 1
            if retry >= max_retry:
                return "GPT3.5 turbo error: %s" % oops
            print('Error communicating with OpenAI:', oops)
            sleep(1)

In [5]:
chadgpt_completion(a, '\n'.join([info["metadata"]["text"] for info in results["matches"]]))

"Malaysia has a tropical climate, which means that it is suitable for growing a wide variety of plants. Some plants that can be grown in Malaysia include:\n1. Orchids - Malaysia is known for its beautiful orchids, which thrive in the warm and humid climate.\n2. Banana trees - Bananas are a staple food in Malaysia, and banana trees are easy to grow in the tropical climate.\n3. Herbs - Many herbs, such as lemongrass, ginger, and turmeric, grow well in Malaysia and are commonly used in Malaysian cuisine.\n4. Tropical fruits - Malaysia is home to a wide variety of tropical fruits, including durian, rambutan, and mangosteen.\n5. Ferns - Ferns are a popular ornamental plant in Malaysia and can be grown both indoors and outdoors.\n6. Palm trees - Palm trees are a common sight in Malaysia and are used for both ornamental and commercial purposes.\n7. Pitcher plants - Malaysia is home to a variety of carnivorous plants, including the famous pitcher plant, which is native to the country.\n8. Bamb

## next question

In [10]:
a= "How to take care of orchids?"

vector = gpt3_embedding(a)
results = vdb.query(vector=vector, top_k=3, include_values=False, include_metadata=True, namespace="casia_kb")
chadgpt_completion(a, '\n'.join([info["metadata"]["text"] for info in results["matches"]]))

Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, your messages resulted in 6099 tokens. Please reduce the length of the messages.
Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, your messages resulted in 6099 tokens. Please reduce the length of the messages.
Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, your messages resulted in 6099 tokens. Please reduce the length of the messages.
Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, your messages resulted in 6099 tokens. Please reduce the length of the messages.


"GPT3.5 turbo error: This model's maximum context length is 4097 tokens. However, your messages resulted in 6099 tokens. Please reduce the length of the messages."

In [15]:
a= "How to take care of orchids?"

vector = gpt3_embedding(a)
results = vdb.query(vector=vector, top_k=2, include_values=False, include_metadata=True, namespace="casia_kb")
chadgpt_completion(a, '\n'.join([info["metadata"]["text"] for info in results["matches"]]))

Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, you requested 4458 tokens (3958 in the messages, 500 in the completion). Please reduce the length of the messages or completion.
Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, you requested 4458 tokens (3958 in the messages, 500 in the completion). Please reduce the length of the messages or completion.
Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, you requested 4458 tokens (3958 in the messages, 500 in the completion). Please reduce the length of the messages or completion.
Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, you requested 4458 tokens (3958 in the messages, 500 in the completion). Please reduce the length of the messages or completion.


"GPT3.5 turbo error: This model's maximum context length is 4097 tokens. However, you requested 4458 tokens (3958 in the messages, 500 in the completion). Please reduce the length of the messages or completion."

In [16]:
a= "How to take care of orchids?"

vector = gpt3_embedding(a)
results = vdb.query(vector=vector, top_k=1, include_values=False, include_metadata=True, namespace="casia_kb")
chadgpt_completion(a, '\n'.join([info["metadata"]["text"] for info in results["matches"]]))

'Orchids are beautiful and delicate plants that require proper care to thrive. Here are some tips for taking care of orchids:\n1. Light: Orchids need bright, indirect light to grow and bloom. Avoid direct sunlight as it can burn the leaves.\n2. Water: Overwatering is the most common mistake people make with orchids. Allow the top 50% of the potting compost to dry out before watering. Check the roots of an orchid plant monthly to be sure they are staying whitish green and plump. Set the plant in a deep saucer of water for about 30 minutes so it can absorb water through the drip holes in the bottom on the pot. Use tepid water that has not passed through a softener (too salty). If your household water has a lot of chemicals in it, allow the water to sit out overnight before using it.\n3. Humidity: Orchids need high humidity, 50% or better. Setting an orchid plant in a stand and placing the stand in a tray filled with pebbles and water helps increase the humidity. Be sure the plant never s

In [19]:
openai.apikey = "--"
a= "How to take care of orchids?"

vector = gpt3_embedding(a)
results = vdb.query(vector=vector, top_k=1, include_values=False, include_metadata=True, namespace="casia_kb")
chadgpt_completion(a, '\n'.join([info["metadata"]["text"] for info in results["matches"]]))

'Orchids are beautiful and delicate plants that require proper care to thrive. Here are some tips for taking care of orchids:\n1. Light: Orchids need bright, indirect light to grow and bloom. Avoid direct sunlight as it can burn the leaves.\n2. Water: Overwatering is the most common mistake people make with orchids. Allow the top 50% of the potting compost to dry out before watering. Check the roots of an orchid plant monthly to be sure they are staying whitish green and plump. Set the plant in a deep saucer of water for about 30 minutes so it can absorb water through the drip holes in the bottom on the pot. Use tepid water that has not passed through a softener (too salty). If your household water has a lot of chemicals in it, allow the water to sit out overnight before using it.\n3. Humidity: Orchids need high humidity, 50% or better. Setting an orchid plant in a stand and placing the stand in a tray filled with pebbles and water helps increase the humidity. Be sure the plant never s

In [21]:
a= "How to take care of orchids?"

vector = gpt3_embedding(a)
results = vdb.query(vector=vector, top_k=2, include_values=False, include_metadata=True, namespace="casia_kb")
chadgpt_completion(a, '\n'.join([info["metadata"]["text"] for info in results["matches"]]))

'Taking care of orchids requires attention to their specific needs. Here are some general guidelines to help you care for your orchids:\n1. Light: Orchids need bright, indirect light. Place them near an east or west-facing window where they can receive about 10 hours of light a day. Avoid direct sunlight as it can damage the leaves.\n2. Watering: Orchids should be watered when the top 50% of the potting compost is dry. Check the roots monthly to ensure they are whitish green and plump. Water the plant by placing it in a deep saucer of water for about 30 minutes, allowing it to absorb water through the drainage holes. Use tepid water that has not passed through a softener. Avoid over-watering, as it can lead to root rot.\n3. Humidity: Orchids thrive in high humidity, around 50% or higher. Increase humidity by placing the orchid on a tray filled with pebbles and water. Ensure the plant does not sit directly in the water.\n4. Fertilizing: Feed your orchid weakly and weekly when it is acti

In [22]:
!pip show openai

Name: openai
Version: 0.27.8
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /Users/farisology/.pyenv/versions/3.11.0/envs/casiaenv/lib/python3.11/site-packages
Requires: aiohttp, requests, tqdm
Required-by: 


In [23]:
a= "How to take care of orchids?"

vector = gpt3_embedding(a)
results = vdb.query(vector=vector, top_k=1, include_values=False, include_metadata=True, namespace="casia_kb")

In [25]:
'\n'.join([info["metadata"]["text"] for info in results["matches"]])

'Orchid – Phalaenopsis Orchid PlantOrchid plants are one of the largest plant species in the world with over 100,000 different varieties, but only small percentage are suitable for growing indoors. A phalaenopsis orchid plant, also known as the Moth Orchid, is one of the varieties that does quite well. There are about 45 species in the\xa0Phalaenopsis\xa0genus and most are native to India, southern China, and Southeast Asia. A phalaenopsis orchid is a type of epiphytic plant which means, in nature, it grows\xa0on the trunks and branches of trees. As\xa0a houseplant, and with proper care, a phalaenopsis orchid plant blooms again and again for many years. It has become very popular because unlike other orchid plants, it has no resting period and may flower several times throughout the year.\nIn nature a phalaenopsis orchid plant sends out aerial roots so it can cling to trees, so as a houseplant\xa0most of its roots are above the pot and grow outside the container. This is perfectly norm